 Use tf115-cpu conda environment
 
Remeber to regenerate the structured_graph for different PC

## Preparation

In [1]:
from model_functions.GNN_model_functions import SocialGNN, get_inputs_outputs
from model_functions.RNN_model_functions import CueBasedLSTM

from collections import namedtuple
import pickle
import os 

from tqdm import tqdm

In [2]:
data_input_dir = '../Data/preprocess/structured_graph'

In [3]:
def load_pickle(path):
    with open(path, 'rb') as f:
        pickled = pickle.load(f)
    return pickled

def save_pickle(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

In [4]:
videos = load_pickle(data_input_dir)

In [5]:
def load_model(model_selection, output_dim, feature_size = None):    
    if model_selection == 'SocialGNN_V':
        model_config = namedtuple('model_config', 'NUM_NODES NUM_AGENTS V_SPATIAL_SIZE E_SPATIAL_SIZE V_TEMPORAL_SIZE V_OUTPUT_SIZE BATCH_SIZE CLASS_WEIGHTS LEARNING_RATE LAMBDA')
        if output_dim == 2:
            config = model_config(NUM_NODES = 5, NUM_AGENTS = 5, V_SPATIAL_SIZE = 12, E_SPATIAL_SIZE = 12, V_TEMPORAL_SIZE = 6, V_OUTPUT_SIZE = 2, BATCH_SIZE = 20, CLASS_WEIGHTS = [1.0, 1.5], LEARNING_RATE = 0.001, LAMBDA = 0.01)
        elif output_dim == 5:
            config = model_config(NUM_NODES = 5, NUM_AGENTS = 5, V_SPATIAL_SIZE = 12, E_SPATIAL_SIZE = 12, V_TEMPORAL_SIZE = 6, V_OUTPUT_SIZE = 5, BATCH_SIZE = 20, CLASS_WEIGHTS = [5.69, 4.42, 1.85, 1.66, 1.0], LEARNING_RATE = 0.001, LAMBDA = 0.01)
        sample_graph_dicts_list, video_timesteps, _ = get_inputs_outputs(videos)
        model = SocialGNN(videos, config, sample_graph_dicts_list)
    
    elif model_selection == 'CueBasedLSTM' or model_selection == 'CueBasedLSTM-Relation':
        if feature_size == None:
            feature_size = (pc+5)*5 if model_selection == 'CueBasedLSTM' else (pc+5)*5+20
        use_edge = False if model_selection == 'CueBasedLSTM' else True
        model_config = namedtuple('model_config', 'FEATURE_SIZE V_TEMPORAL_SIZE V_OUTPUT_SIZE BATCH_SIZE CLASS_WEIGHTS LEARNING_RATE LAMBDA')
        if output_dim == 2:
            config = model_config(FEATURE_SIZE = feature_size, V_TEMPORAL_SIZE = 6, V_OUTPUT_SIZE = 2, BATCH_SIZE = 20, CLASS_WEIGHTS = [1.0, 1.5], LEARNING_RATE = 0.001, LAMBDA = 0.01)
        elif output_dim == 5:
            config = model_config(FEATURE_SIZE = feature_size, V_TEMPORAL_SIZE = 6, V_OUTPUT_SIZE = 5, BATCH_SIZE = 20, CLASS_WEIGHTS = [5.69, 4.42, 1.85, 1.66, 1.0], LEARNING_RATE = 0.001, LAMBDA = 0.01)
        model = CueBasedLSTM(videos, config, explicit_edges=use_edge)
    else:
        raise NotImplementedError
    return model

In [6]:
def save_layer_representations(paths, feature_size = None):
    for model_selection, input_model_paths in paths.items():
        if input_model_paths:
            print(f'Processing {model_selection}')
            model = load_model(model_selection, output_size, feature_size=feature_size)
            for input_path, output_path in tqdm(input_model_paths):
                model.load_model(input_path)
                layer_representations = model.get_layer_representations()
                for layer in layer_representations:
                    output_layer_path = os.path.join(output_path, f'{layer}.pkl')
                    save_pickle(layer_representations[layer], output_layer_path)

## get representations

In [20]:
pc = 90

In [8]:
def print_paths(paths):
    print('These are the input models:')
    for key, value in paths.items():
        print(f'{len(value)} {key} models')
        for i in value:
            print(f'input: {i[0]}')
            print(f'output: {i[1]}')

In [9]:
## Original Models
output_size = 2
model_dir = f'../retrained_models'
output_dir = f'../layer_representations'
paths = {
    'CueBasedLSTM': [], 
    'SocialGNN_V': [], 
    'CueBasedLSTM-Relation': []
}

for option in paths.keys():
    input_model_class_path = os.path.join(model_dir, f'PC{pc}', option)
    output_model_class_path = os.path.join(output_dir, f'PC{pc}', option)
    for parameter in os.listdir(input_model_class_path):
        input_model_path = os.path.join(input_model_class_path, parameter)
        output_model_path = os.path.join(output_model_class_path, parameter)
        os.makedirs(output_model_path, exist_ok=True)
        paths[option].append([input_model_path, output_model_path])

print_paths(paths)


These are the input models:
20 CueBasedLSTM models
input: ../retrained_models\PC20\CueBasedLSTM\2_14Jun_0_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations\PC20\CueBasedLSTM\2_14Jun_0_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models\PC20\CueBasedLSTM\2_14Jun_1_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations\PC20\CueBasedLSTM\2_14Jun_1_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models\PC20\CueBasedLSTM\2_14Jun_2_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations\PC20\CueBasedLSTM\2_14Jun_2_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models\PC20\CueBasedLSTM\2_14Jun_3_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations\PC20\CueBasedLSTM\2_14Jun_3_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models\PC20\CueBasedLSTM\2_14Jun_4_CueBasedLSTM_125_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations\PC20\CueBa

In [10]:
save_layer_representations(paths)

Processing CueBasedLSTM
.............DEFINING INPUT PLACEHOLDERS..............

.............BUILDING GRAPH..............

.............INITIALIZATION SESSION..............


100%|██████████| 20/20 [00:24<00:00,  1.20s/it]


Processing SocialGNN_V
.............DEFINING INPUT PLACEHOLDERS..............

.............BUILDING GRAPH..............

.............INITIALIZATION SESSION..............


100%|██████████| 20/20 [08:00<00:00, 24.05s/it]


Processing CueBasedLSTM-Relation
.............DEFINING INPUT PLACEHOLDERS..............

.............BUILDING GRAPH..............

.............INITIALIZATION SESSION..............


100%|██████████| 20/20 [01:28<00:00,  4.44s/it]


## ONLY EDGE MODELS

In [21]:
def construct_paths(f_paths, f_option, f_model_class, f_model_dir, f_output_dir):
    f_input_model_class_path = os.path.join(f_model_dir, f_model_class)
    f_output_model_class_path = os.path.join(f_output_dir, f_model_class)
    for f_parameter in os.listdir(f_input_model_class_path):
        f_input_model_path = os.path.join(f_input_model_class_path, f_parameter)
        f_output_model_path = os.path.join(f_output_model_class_path, f_parameter)
        os.makedirs(f_output_model_path, exist_ok=True)
        f_paths[f_option].append([f_input_model_path, f_output_model_path])
    return f_paths

In [22]:
output_size = 2
option = 'SocialGNN_V'
model_dir = '../retrained_models/only_edge_models'
output_dir = f'../layer_representations/only_edge_models/'
paths = {'SocialGNN_V': []}

In [ ]:
# Randomized GNN
model_class = f'SocialGNN_randomize_node_PC{pc}'
new_paths = construct_paths(paths, option, model_class, model_dir, output_dir)
print_paths(new_paths)

import random
def randomize_sequence(sequences):
    for seq_idx in range(len(sequences)):
        for frame_idx in range(len(sequences[seq_idx]['graph_dicts'])):
            sequences[seq_idx]['graph_dicts'][frame_idx]['nodes'] =  [[random.uniform(-20, 20) for _ in range(pc+5)] for _ in range(5)]
    return sequences

videos = randomize_sequence(load_pickle(data_input_dir))
save_layer_representations(new_paths)


These are the input models:
20 SocialGNN_V models
input: ../retrained_models/only_edge_models\SocialGNN_randomize_node_PC20\2_14Jun_0_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/SocialGNN_randomize_node_PC20\2_14Jun_0_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\SocialGNN_randomize_node_PC20\2_14Jun_1_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/SocialGNN_randomize_node_PC20\2_14Jun_1_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\SocialGNN_randomize_node_PC20\2_14Jun_2_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/SocialGNN_randomize_node_PC20\2_14Jun_2_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\SocialGNN_randomize_node_PC20\2_14Jun_3_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: 

 10%|█         | 2/20 [03:46<33:58, 113.27s/it]

In [15]:
# Zeroed GNN
paths = {'SocialGNN_V': []}
model_class = f'SocialGNN_zero_node_PC{pc}'
new_paths = construct_paths(paths, option, model_class, model_dir, output_dir)
print_paths(new_paths)

import random
def zero_sequence(sequences):
    for seq_idx in range(len(sequences)):
        for frame_idx in range(len(sequences[seq_idx]['graph_dicts'])):
            sequences[seq_idx]['graph_dicts'][frame_idx]['nodes'] =  [[0.0 for _ in range(pc+5)] for _ in range(5)]
    return sequences     

videos = zero_sequence(load_pickle(data_input_dir))
save_layer_representations(new_paths)


These are the input models:
20 SocialGNN_V models
input: ../retrained_models/only_edge_models\SocialGNN_zero_node_PC20\2_14Jun_0_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/SocialGNN_zero_node_PC20\2_14Jun_0_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\SocialGNN_zero_node_PC20\2_14Jun_1_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/SocialGNN_zero_node_PC20\2_14Jun_1_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\SocialGNN_zero_node_PC20\2_14Jun_2_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/SocialGNN_zero_node_PC20\2_14Jun_2_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\SocialGNN_zero_node_PC20\2_14Jun_3_SocialGNN_V_5_5_12_12_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_

100%|██████████| 20/20 [15:59<00:00, 47.98s/it]


In [16]:
# for LSTM only edge models
output_size = 2
option = 'CueBasedLSTM-Relation'
model_dir = '../retrained_models/only_edge_models'
output_dir = f'../layer_representations/only_edge_models/'
pc = 0
model_class = f'LSTM_only_edge'

paths = {'CueBasedLSTM-Relation': []}
new_paths = construct_paths(paths, option, model_class, model_dir, output_dir)
print_paths(new_paths)

def remove_node(sequences):
    for seq_idx in range(len(sequences)):
        for frame_idx in range(len(sequences[seq_idx]['graph_dicts'])):
            sequences[seq_idx]['graph_dicts'][frame_idx]['nodes'] =  [[] for _ in range(5)]
    return sequences
videos = remove_node(load_pickle(data_input_dir))
save_layer_representations(paths, feature_size = 20)


These are the input models:
20 CueBasedLSTM-Relation models
input: ../retrained_models/only_edge_models\LSTM_only_edge\2_14Jun_0_20_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/LSTM_only_edge\2_14Jun_0_20_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\LSTM_only_edge\2_14Jun_1_20_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/LSTM_only_edge\2_14Jun_1_20_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\LSTM_only_edge\2_14Jun_2_20_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/LSTM_only_edge\2_14Jun_2_20_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\LSTM_only_edge\2_14Jun_3_20_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/only_edge_models/LSTM_only_edge\2_14Jun_3_20_6_2_20_1.0_1.5_0.001_0.01
input: ../retrained_models/only_edge_models\LSTM_only_edge\2_14Jun_4_20_6_2_20_1.0_1.5_0.001_0.01
output: ../layer_representations/o

100%|██████████| 20/20 [05:08<00:00, 15.43s/it]
